# 5. KoBERT

1. 환경 설정 및 라이브러리 로딩
2. 데이터 로딩 및 전처리 : KorCCVi_v2.csv 파일 로드, 텍스트 데이터 정제
3. 토크나이저 및 데이터셋 준비 : KoBERT에 맞는 토크나이저를 사용하여 데이터를 토큰화하고, 학습에 필요한 데이터셋 생성
4. 모델 정의 및 학습 : KoBERT 기반의 분류 모델을 정의, 학습 진행
5. 모델 평가 및 예측 : 테스트 데이터를 사용하여 모델의 성능 평가 및 예측 결과 확인

### 📌 KoBERT를 활용한 보이스 피싱 탐지 모델 구축 코드
- 참고: https://github.com/selfcontrol7/Korean_Voice_Phishing_Detection
- 주요 목적: 통화 텍스트에서 보이스 피싱(1) vs 정상(0) 판별
- 추가 기능: 중요 키워드 추출

## [1] 환경 설정 및 라이브러리 로딩

In [ ]:
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
#pip uninstall tensorflow tf-keras tensorflow-estimator keras keras-preprocessing -y

In [ ]:
#!pip uninstall transformers -y

In [ ]:
#!pip install transformers[torch]

In [ ]:
from transformers import BertModel, BertTokenizer

In [ ]:
#pip install gluonnlp==0.10.0 --no-build-isolation --no-deps

In [ ]:
#pip install kobert-transformers

In [ ]:
#pip uninstall numpy -y

In [ ]:
#pip install numpy==1.26.4

In [ ]:
# HugginFace 관련 시각화 위젯 설치
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [1]:
# 🔧 1. 라이브러리 임포트
import torch  # GPU 사용을 위한 PyTorch
from torch import nn  # 신경망 모델 구성용
from torch.utils.data import Dataset, DataLoader  # 데이터셋 및 배치 구성

from transformers import BertTokenizer, BertModel  # HuggingFace용 KoBERT 모델
from kobert_transformers import get_tokenizer, get_kobert_model  # KoBERT용 전용 함수

import numpy as np  # 수치 연산
import pandas as pd  # 데이터프레임 처리
from sklearn.model_selection import train_test_split  # 학습/테스트 분리
from tqdm import tqdm  # 진행바 시각화
from konlpy.tag import Okt  # 키워드 추출을 위한 형태소 분석기
from torch.optim import AdamW # HuggingFace transformers에서 import

C:\downloads\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


## [2] 데이터 로딩 및 전처리

In [2]:
# 데이터 임포트
# CSV 파일 경로
csv_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/phishing_dataset/KorCCVi_dataset/KorCCViD_v1.3_fullcleansed.csv'

In [3]:
# CSV 파일을 읽고 데이터를 무작위로 섞은 후 인덱스를 리셋함
dataset = pd.read_csv(csv_path).sample(frac=1.0).reset_index(drop=True)

In [4]:
# 'Transcript' 컬럼은 통화 텍스트, 'Label' 컬럼은 피싱 여부 라벨 (0: 정상, 1: 피싱)
texts = dataset['Transcript'].tolist()  # 텍스트 데이터를 리스트로 추출
labels = dataset['Label'].tolist()      # 라벨 데이터를 리스트로 추출

In [5]:
# 🔍 미리보기: 텍스트와 라벨 5개 출력
print("📝 텍스트 샘플:", texts[:5])
print("🏷️ 라벨 샘플:", labels[:5])

📝 텍스트 샘플: ['너무 연애 어서 그런 일시 감정 으로 연애 다가 나중 괜히 자신 괴롭히 아닌가 그런 생각 어서 연애 아직 다는 생각 그렇게 지난번 타로 카드 지만 보고 연애 다고 올해 그냥 공부 라고 했잖아 근데 들으니까 그냥 그냥 연애 공부 하자라 생각 올해', '그러고 거기 너무 추운데 날씨 차라리 여름 모르 그서 날씨 차라리 홍콩 아님 저기 마닐라 그쪽 으로 홍콩 으로 결제 지금 결정 사실 지금 모르 갈지 갈지 친구 원래 같이 다음 친구 약속 려고 처럼 보여 혼자 라도 올까 지금 고민 인데 거기 아님 라도', '요즘 장기 힘들 기본 전부 개월 개월 잖아 개월 개월 아님 개월 개월 인데 글쎄 내년 내년도 아무래도 학교 복학 다음 여러 아무래도 다른 학교 면은 힘들 그래서 최근 다양 알바 통해서 사회 경험 으려 사람 마음 데로', '그래도 재수 재수 당연히 재수 근데 동안 진짜 어떻게 버텼 모르 근데 열심히 아니 거기 사람 열심히 잖아 진짜 그냥 여기 열심히 사람 저기 열심히 사람 그러니까 되게 그냥 뭔가 진짜 그냥 다른 세계 에서 그냥 그냥 그냥 완전 다른 사람 으로 다른 세계 에서 느낌', '여보세요 여보세요 드십니까 서울 중앙 지검 생각 합니다 통화 십니까 그거 까지 문희상 때문 연락 드렸 고요 선생 혹시 김용술 사람 계십니까 그냥 모르 사람 고요 다름 아니 15 일자 해서 용인 김용수 일단 여섯 금융 범죄 살리실산 광고 현장 에서 대량 대포통장 어떻게 지내 서울 중앙 지방 검찰청 김용 범죄 수사 학과 수사관 입니다 에요 다시 레시피 내용 사건']
🏷️ 라벨 샘플: [0, 0, 0, 0, 1]


## [3] 훈련 / 테스트 데이터 분리

| 구성 요소              | 설명                               |
| ------------------ | -------------------------------- |
| `texts`            | 전체 통화 텍스트 (입력 문장 리스트)            |
| `labels`           | 각 텍스트에 대한 보이스 피싱 여부 레이블 (0 또는 1) |
| `train_test_split` | sklearn의 함수로 데이터를 훈련/테스트 세트로 나눔  |
| `test_size=0.2`    | 20%는 테스트용, 80%는 훈련용으로 분할         |
| `random_state=42`  | 실행 시마다 같은 결과를 얻기 위한 시드 설정        |


In [11]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

In [13]:
print("🔍 훈련용 예시:", train_texts[:3])
print("🧪 테스트용 예시:", test_texts[:3])

🔍 훈련용 예시: ['정부 정책 자금 대출 신청 대리 신청 때문 본인 확인 대한 정보 일치 접수 구요 저희 무료 신용 정보 통해서 조회 등록 드리 위해 주민 번호 자리 확인 부탁 드리 습니다 그럼 본인 확인 위해서 문자 숫자 여섯 자리 발송 드릴 건데요 요즘 개인 정보 유출 도용 방지 위해서 임시 아이피 발급 통해서 전하 본인 확인 도와 드리', '에서 취급 일반 신용 대출 담보 대출 아니 구요 한국 자산 관리 공사 정부 정책 자금 입니다 한국 자산 관리 공사 여기 최종 최종 승인 나셔 여기 심사 거기 최종 승인 으신 고객 께서 대출금 고객 근처 중앙 내방 셔서 자금 수령 고요 우선 접수 통해서 서민 금융 통합 지원 센터 접수 세요 거기 배정 접수 시켜 드리 면은 거기 대출 전문 담당자 배정 으실 겁니다 고객 께서 정부 정책 자금 정부 에서 자금 구요 정부 지원 저희 접수처 고객', '다른 일단 그냥 뭔가 공기 다른 되게 그냥 서울 되게 자주 지만 마다 되게 들뜨 뭔가 되게 그냥 와도 느낌 그냥 그대로 느낌 그런데 되게 괜찮 일단 동네 수원 일단 사람 너무 우리 우리 잘못 하나 보이 면은 바로 생기 그러니까 되게 서울 괜찮 으로 자주 자주']
🧪 테스트용 예시: ['그렇 농협 국민 인가요 농협 경우 니까 도개 형성 자체 어떤 용도 계좌 습니까 어서 적금 청약 마이너스 통장 자유 입출금 기타 등등 어떤 으로 형성 으시 자율 출근 하나 밖에 형성 나요 월급 통장 자율 지금 농협 에서 으세요 농협 하나 밖에 으시 고요 국민은행 요네 적금 국민은행 경우 그렇 다면 본인 께서 금융 저축 CMA 계좌 본인 명의 부분 으십니까 그런 으시 통장 10 실제로 금액 들어오 셔서 피해 계세요 본인 마지막 까지 거래 잔액 얼마 정도 본인 계좌 얼마 정도 자금 형성 으며 본인 계좌 간략 100 100 미만 500 500 천만 천만 100 까지 간략 지수 부탁', '근까 그렇게 느낀 근데 막상 사귀 우리 얘기 얘기 니까 그제서야 갑자기 자기 그게 아니 다는 자기 그러 미안 지만 애정 

## [4] KoBERT용 토크나이저 및 데이터셋 구성

| 객체 이름           | 역할 요약                                              | 세부 설명                                      |
| --------------- | -------------------------------------------------- | ------------------------------------------ |
| `train_dataset` | 학습용 원본 데이터셋                                        | 토큰화된 입력과 라벨을 포함. `BERTDataset` 클래스를 통해 구성됨 |
| `test_dataset`  | 테스트용 원본 데이터셋                                       | 학습이 아닌 평가 목적의 데이터셋                         |
| `train_loader`  | 학습용 데이터 배치 처리기 (`batch_size=16`, `shuffle=True`)   | 에폭마다 다른 순서로 섞어서 학습을 안정적으로 수행               |
| `test_loader`   | 테스트용 데이터 배치 처리기 (`batch_size=16`, `shuffle=False`) | 평가 시 순서를 유지하여 성능 측정                        |


In [15]:
# ✅ KoBERT 전용 토크나이저 로딩 (skt/kobert-base-v1에 맞춰진 사전 설정 포함)
kobert_tokenizer = get_tokenizer()

# ✅ KoBERT 사전학습 모델 로딩 (Transformer 기반 BERT 모델)
kobert_model = get_kobert_model()

# 📏 입력 토큰 최대 길이 설정 (128 토큰 이상은 자르고, 부족하면 패딩)
MAX_LEN = 128

In [17]:
# 📦 PyTorch Dataset 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts                  # 입력 텍스트 리스트 (문장들)
        self.labels = labels                # 해당 문장의 라벨 리스트 (0 or 1)
        self.tokenizer = tokenizer          # KoBERT용 토크나이저
        self.max_len = max_len              # 최대 길이 설정

    def __len__(self):
        return len(self.texts)              # 전체 데이터 길이 반환

    def __getitem__(self, idx):
        # 🔠 텍스트 하나를 토크나이징하여 KoBERT에 맞는 포맷으로 변환
        encoded = self.tokenizer(
            self.texts[idx],                # 현재 인덱스의 텍스트
            padding='max_length',           # 부족한 길이는 패딩
            truncation=True,                # 초과한 길이는 자름
            max_length=self.max_len,        # 최대 길이 설정
            return_tensors='pt'             # PyTorch Tensor로 반환
        )
        # 🧷 배치로 만들기 위해 차원을 줄임 (squeeze(0): batch dimension 제거)
        item = {key: val.squeeze(0) for key, val in encoded.items()}

        # 🎯 정답 라벨도 텐서로 변환하여 추가
        item['labels'] = torch.tensor(self.labels[idx])
        return item                         # input_ids, attention_mask, token_type_ids, labels 포함

In [19]:
# 🧪 학습용 데이터셋 생성
train_dataset = BERTDataset(train_texts, train_labels, kobert_tokenizer, MAX_LEN)
# 🧪 테스트용 데이터셋 생성
test_dataset = BERTDataset(test_texts, test_labels, kobert_tokenizer, MAX_LEN)
# 🚚 학습용 데이터로더: 배치 단위로 데이터를 불러오며 무작위 셔플
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# 🚚 테스트용 데이터로더: 배치 단위로 데이터를 불러오되 셔플은 하지 않음
test_loader = DataLoader(test_dataset, batch_size=16)

In [21]:
# 데이터셋/로더 구조 미리보기 (1배치만 확인)
print("🔍 학습 데이터셋 예시:")
for batch in train_loader:
    print("📦 input_ids:", batch['input_ids'].shape)
    print("📦 attention_mask:", batch['attention_mask'].shape)
    print("📦 token_type_ids:", batch['token_type_ids'].shape)
    print("🏷️ labels:", batch['labels'].shape)
    break  # 한 배치만 확인

print("\n🔍 테스트 데이터셋 예시:")
for batch in test_loader:
    print("📦 input_ids:", batch['input_ids'].shape)
    print("📦 attention_mask:", batch['attention_mask'].shape)
    print("📦 token_type_ids:", batch['token_type_ids'].shape)
    print("🏷️ labels:", batch['labels'].shape)
    break  # 한 배치만 확인


🔍 학습 데이터셋 예시:
📦 input_ids: torch.Size([16, 128])
📦 attention_mask: torch.Size([16, 128])
📦 token_type_ids: torch.Size([16, 128])
🏷️ labels: torch.Size([16])

🔍 테스트 데이터셋 예시:
📦 input_ids: torch.Size([16, 128])
📦 attention_mask: torch.Size([16, 128])
📦 token_type_ids: torch.Size([16, 128])
🏷️ labels: torch.Size([16])


In [23]:
# 첫 배치에서 첫 번째 샘플 문장을 디코딩해서 보기
for batch in train_loader:
    input_ids = batch['input_ids'][0]  # 첫 샘플 선택
    label = batch['labels'][0].item()  # 라벨도 함께 출력

    # 디코딩: 토큰 ID → 문장
    decoded_text = kobert_tokenizer.decode(input_ids, skip_special_tokens=True)

    print("📝 디코딩된 문장:", decoded_text)
    print("🏷️ 라벨 (0: 정상, 1: 피싱):", label)
    break  # 한 번만 실행

📝 디코딩된 문장: 말씀 세요 업무 나요 그냥 해지 시간 말씀 어요 해지 시간 다시 말씀 어요 고요 일단 지금 본인 본인 입출금 으로 자금 건가요 본인 입출금 으로 지금 예금 잔액 으시 잖아요 예금 잔액 본인 입출금 으로 구요 본인 지금 입출금 으로 잔액 어떻게 지금 1200 1200 으시 일단 지금 본인 으셨고 우리 방문 신데 동소 시간 어떤 얼마나 걸리 어요 10 정도 도보 10 정도 걸리 일단 우리 방문 셔야 여보세요 세요 지금 벌써 세요 지금
🏷️ 라벨 (0: 정상, 1: 피싱): 1


## [5] KoBERT 분류 모델 정의

| 구성 요소             | 설명                                                                      |
| ----------------- | ----------------------------------------------------------------------- |
| `bert_model`      | `get_kobert_model()`을 통해 불러온 사전학습된 KoBERT 모델. 입력 문장을 BERT 인코딩하여 벡터로 변환  |
| `hidden_size`     | BERT의 출력 차원 크기 (기본: 768). 이 값은 Linear 레이어의 입력 크기로 사용됨                   |
| `num_classes`     | 분류 클래스 수 (보이스피싱은 0/1이므로 2)                                              |
| `dr_rate`         | Dropout 비율. 학습 시 과적합을 방지하기 위해 일부 뉴런을 무작위로 끄는 확률 (예: 0.3 = 30%)          |
| `self.classifier` | 선형 계층 (`nn.Linear`)로 \[CLS] 토큰 벡터를 이진 분류 확률값으로 변환                       |
| `self.dropout`    | Dropout 레이어. 학습 중 과적합을 줄이기 위해 중간 벡터 일부를 무작위로 제거                         |
| `self.softmax`    | Linear 출력값을 0\~1 사이의 확률값으로 변환. 두 클래스의 확률 분포로 해석 가능                      |
| `forward()`       | 학습 또는 추론 시 호출되는 메서드. BERT → \[CLS] → Dropout → Linear → Softmax 순으로 처리됨 |
   |


## [ 모델 ! ]

In [25]:
class KoBERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size=768, num_classes=2, dr_rate=0.3):
        """
        bert_model: get_kobert_model()로 로딩한 KoBERT 모델
        hidden_size: BERT의 hidden layer 크기 (기본 768)
        num_classes: 분류 클래스 수 (0: 정상, 1: 피싱 → 2개)
        dr_rate: dropout 비율
        """
        super(KoBERTClassifier, self).__init__()
        self.bert = bert_model
        self.dr_rate = dr_rate

        # ✅ 분류 레이어 정의
        self.classifier = nn.Linear(hidden_size, num_classes)

        # ✅ 드롭아웃 설정
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

        # ✅ 확률 출력을 위한 Softmax
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        """
        BERT 입력값 처리 → Dropout → Linear 분류 → Softmax
        """
        # BERT 모델 실행
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        
        # [CLS] 토큰의 출력값 사용 (문장 요약 벡터)
        cls_output = outputs.pooler_output

        # Dropout 적용 (training 시에만 활성화됨)
        if self.dr_rate:
            cls_output = self.dropout(cls_output)

        # Linear 분류 → Softmax 확률 출력
        logits = self.classifier(cls_output)
        return self.softmax(logits)


-------------------------------------------------------------------------------

## [6] Optimizer + 학습률 스케줄러 설정

In [27]:
# ✅ 장치 설정: GPU 사용 가능하면 GPU 사용, 아니면 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 모델 인스턴스 생성 코드 : 
- 실제 학습에 사용된 model 객체 생성
- 이후 학습 / 검증 과정에서 이 모델 사용

In [29]:
# ✅ KoBERT 분류 모델 생성 (드롭아웃 포함)
model = KoBERTClassifier(
    bert_model=kobert_model,  # 사전학습 KoBERT 모델
    hidden_size=768,          # hidden state 차원 크기 (기본값)
    num_classes=2,            # 분류 클래스 수 (0: 정상, 1: 피싱)
    dr_rate=0.3               # Dropout 비율 설정
).to(device)  # 모델을 GPU 또는 CPU로 할당

-----------------------------------------------------------------------

In [31]:
# ✅ weight decay를 다르게 적용하기 위한 파라미터 분리
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        'weight_decay': 0.01  # 일반 파라미터에는 가중치 감쇠 적용
    },
    {
        'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        'weight_decay': 0.0   # bias나 LayerNorm에는 감쇠 미적용
    }
]

In [33]:
# ✅ 옵티마이저 설정 (AdamW)
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

In [35]:
# ✅ 손실 함수 정의 (Cross Entropy)
loss_fn = nn.CrossEntropyLoss()

In [39]:
from transformers import get_cosine_schedule_with_warmup

In [41]:
# ✅ 학습 스케줄 설정
EPOCHS = 3
warmup_ratio = 0.1  # 전체 학습 스텝 중 워밍업 비율

# 전체 학습 스텝 수 계산 (배치 수 × epoch 수)
t_total = len(train_loader) * EPOCHS
warmup_step = int(t_total * warmup_ratio)

# 코사인 학습률 스케줄러 설정
scheduler = get_cosine_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total
)

In [43]:
# ✅ 정확도 계산 함수 (accuracy)
def calc_accuracy(preds, labels):
    _, predicted = torch.max(preds, 1)  # 각 배치에서 확률이 가장 높은 클래스 선택
    correct = (predicted == labels).sum().item()
    return correct / labels.size(0)  # 전체 중 맞춘 비율 반환

In [45]:
# ✅ 정밀도, 재현율, F1-score 계산 함수
# → 모델의 평가 지표로 활용됨
def get_metrics(preds, labels, threshold=0.5):
    pred_classes = torch.argmax(preds, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    tp = ((pred_classes == 1) & (labels == 1)).sum()
    fp = ((pred_classes == 1) & (labels == 0)).sum()
    fn = ((pred_classes == 0) & (labels == 1)).sum()
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return {"precision": precision, "recall": recall, "f1": f1}

In [47]:
# ✅ 평가 함수 (정확도 + 정밀도 + 재현율 + F1-score 출력)
@torch.no_grad()
def evaluate(model, data_loader):
    model.eval()
    total_correct = 0
    total = 0
    all_preds = []
    all_labels = []
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        preds = model(input_ids, attention_mask, token_type_ids)
        total_correct += (preds.argmax(1) == labels).sum().item()
        total += labels.size(0)
        all_preds.append(preds)
        all_labels.append(labels)

    acc = total_correct / total
    all_preds_tensor = torch.cat(all_preds, dim=0)
    all_labels_tensor = torch.cat(all_labels, dim=0)
    metrics = get_metrics(all_preds_tensor, all_labels_tensor)

    print(f"🎯 정확도: {acc*100:.2f}% | 정밀도: {metrics['precision']:.2f} | 재현율: {metrics['recall']:.2f} | F1: {metrics['f1']:.2f}")

## [7] KoBERT 모델 학습 루프

| 구성 요소              | 설명                            |
| ------------------ | ----------------------------- |
| `model.train()`    | 모델을 학습 모드로 전환 (dropout 등 활성화) |
| `loss.backward()`  | 손실 함수로부터 gradient 계산          |
| `optimizer.step()` | 계산된 gradient로 가중치 업데이트        |
| `scheduler.step()` | 학습률 조절 스케줄러 업데이트              |
| `calc_accuracy()`  | 배치 단위 정확도 계산                  |


In [49]:
for epoch in range(EPOCHS):
    model.train()  # 모델을 학습 모드로 전환
    total_loss = 0  # 손실 누적 변수 초기화
    total_acc = 0   # 정확도 누적 변수 초기화

    # 학습 데이터 배치 단위로 반복
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        # 입력값 GPU/CPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        # 예측 수행
        preds = model(input_ids, attention_mask, token_type_ids)

        # 손실 계산
        loss = loss_fn(preds, labels)

        # 옵티마이저 초기화 → 역전파 → 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        # 누적 손실과 정확도 계산
        total_loss += loss.item()
        total_acc += calc_accuracy(preds, labels)

    # 평균 손실과 정확도 출력
    avg_loss = total_loss / len(train_loader)
    avg_acc = total_acc / len(train_loader)
    print(f"[Epoch {epoch+1}] 평균 손실: {avg_loss:.4f} | 평균 정확도: {avg_acc*100:.2f}%")

Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████| 61/61 [18:58<00:00, 18.67s/it]


[Epoch 1] 평균 손실: 0.5179 | 평균 정확도: 81.25%


Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████| 61/61 [19:15<00:00, 18.94s/it]


[Epoch 2] 평균 손실: 0.3307 | 평균 정확도: 99.18%


Epoch 3: 100%|█████████████████████████████████████████████████████████████████████████| 61/61 [19:19<00:00, 19.00s/it]

[Epoch 3] 평균 손실: 0.3267 | 평균 정확도: 99.17%


### Epoch 3 -> 정확도 상승

| 에포크     | 평균 손실 (`loss`) | 평균 정확도 (`accuracy`) | 의미                          |
| ------- | -------------- | ------------------- | --------------------------- |
| Epoch 1 | 0.5027         | 84.31%              | 첫 학습 시작 → 모델이 대략적으로 분류를 시작함 |
| Epoch 2 | 0.3491         | 97.75%              | 모델이 피싱/정상 분류를 잘 학습함         |
| Epoch 3 | 0.3285         | 99.39%              | 거의 완벽에 가까운 성능으로 학습됨         |


## [8] 모델 평가 (테스트 정확도, 정밀도, 재현율, F1-score 측정)

| 항목              | 설명                      |
| --------------- | ----------------------- |
| 정확도 (Accuracy)  | 전체 예측 중 맞춘 비율           |
| 정밀도 (Precision) | 피싱이라고 예측한 것 중 실제 피싱 비율  |
| 재현율 (Recall)    | 실제 피싱 중에서 얼마나 잘 찾아냈는가   |
| F1-score        | 정밀도와 재현율의 조화 평균 (균형 측정) |


In [52]:
model.eval()  # 평가 모드 전환 (Dropout 등 비활성화)

total_correct = 0
total = 0
all_preds = []
all_labels = []

In [54]:
with torch.no_grad():  # 평가 시에는 그래디언트 계산하지 않음
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        preds = model(input_ids, attention_mask, token_type_ids)
        total_correct += (preds.argmax(1) == labels).sum().item()
        total += labels.size(0)

        all_preds.append(preds)
        all_labels.append(labels)

In [55]:
# 전체 정확도 계산
acc = total_correct / total

In [56]:
# 예측 및 정답 텐서를 하나로 합침
all_preds_tensor = torch.cat(all_preds, dim=0)
all_labels_tensor = torch.cat(all_labels, dim=0)

In [57]:
# 정밀도, 재현율, F1 계산
metrics = get_metrics(all_preds_tensor, all_labels_tensor)

In [58]:
# 결과 출력
print(f"\n📊 [테스트 결과]")
print(f"🎯 정확도: {acc * 100:.2f}%")
print(f"📌 정밀도: {metrics['precision']:.2f}")
print(f"📌 재현율: {metrics['recall']:.2f}")
print(f"📌 F1-score: {metrics['f1']:.2f}")


📊 [테스트 결과]
🎯 정확도: 99.59%
📌 정밀도: 1.00
📌 재현율: 0.99
📌 F1-score: 1.00


## [9] 주요 키워드 추출 (형태소 분석 기반)

In [ ]:
from konlpy.tag import Okt
import pandas as pd
import random

In [ ]:
# 🔍 형태소 분석기 준비
okt = Okt()

# ✅ CSV 파일 경로
csv_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/phishing_dataset/KorCCVi_dataset/KorCCViD_v1.3_fullcleansed.csv'

# ✅ CSV 불러오고 무작위 셔플
dataset = pd.read_csv(csv_path).sample(frac=1.0).reset_index(drop=True)

# ✅ 텍스트 컬럼만 추출
texts = dataset['Transcript'].tolist()

In [ ]:
# ✅ 주요 키워드 추출 함수
def extract_keywords(text, top_n=5):
    nouns = okt.nouns(text)
    freq = pd.Series(nouns).value_counts()
    return freq.head(top_n).to_dict()

In [ ]:
# ✅ 랜덤으로 5개 추출
sample_texts = random.sample(texts, 5)

In [ ]:
# ✅ 각 샘플에 대해 키워드 추출
for i, text in enumerate(sample_texts, 1):
    keywords = extract_keywords(text)
    print(f"\n🗂️ 샘플 {i}: {text}")
    print(f"📌 주요 키워드: {keywords}")

## [10] 크롤링한 데이터 모델에 넣어서 예측

In [64]:
import pandas as pd
import torch

# ✅ 데이터 로드: STT 텍스트가 포함된 CSV 파일 경로 지정
csv_path = 'D:/2025_work/2025_VoicePhshing_Detection_Model/dataset/stt_transcripts_sampled.csv'

# ✅ CSV 파일을 pandas로 읽어옴
df = pd.read_csv(csv_path)

# ✅ 데이터프레임에서 상위 2개 행만 샘플로 추출
sampled_rows = df.head(2)

# ✅ 모델을 평가 모드로 전환 (Dropout, BatchNorm 비활성화)
model.eval()

KoBERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [66]:
# ✅ 예측 함수 정의: text를 입력받아 label(0 or 1)과 확신도(score) 반환
def predict_phishing(text, model, tokenizer, max_len=128):
    """
    주어진 text를 KoBERT 모델로 예측하여
    보이스 피싱(1) 또는 정상(0) 여부를 판단하는 함수
    """
    # ✅ 토크나이저로 입력 문장 인코딩 (KoBERT 포맷)
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'  # PyTorch 텐서 형태로 반환
    )

    # ✅ 각 입력을 GPU 또는 CPU로 이동
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    token_type_ids = encoded['token_type_ids'].to(device)

    # ✅ 추론: 그래디언트 계산 없이 모델 예측 수행
    with torch.no_grad():
        output = model(input_ids, attention_mask, token_type_ids)

        # ✅ 예측 결과 중 확률이 가장 높은 클래스 선택 (0: 정상, 1: 피싱)
        pred_label = torch.argmax(output, dim=1).item()

        # ✅ 해당 클래스의 확률값 (0~1) 추출
        confidence = output[0][pred_label].item()

    return pred_label, confidence

In [68]:
# ✅ 샘플 텍스트 각각에 대해 예측 수행
for idx, row in sampled_rows.iterrows():
    text = row['text']        # 통화 내용 텍스트
    true_label = row['label'] # 실제 라벨 (0 또는 1)

    # ✅ 모델을 이용해 예측 수행
    pred_label, confidence = predict_phishing(text, model, kobert_tokenizer)

    # ✅ 예측 결과 출력
    print(f"\n📞 샘플 {idx+1}")
    print(f"▶ 원본 텍스트: {text}")
    print(f"✅ 실제 라벨: {'보이스 피싱' if true_label == 1 else '정상 통화'}")
    print(f"🔍 예측 라벨: {'보이스 피싱' if pred_label == 1 else '정상 통화'} ({confidence*100:.2f}% 확신도)")



📞 샘플 1
▶ 원본 텍스트: 안녕하세요. 공공은행입니다. 신용대출을 받을 수 있을까요? 본인이 맞는지 먼저 확인하겠습니다. 네. 안내 멘트가 나오면 생년월일 6자리를 눌러주세요. 네 알겠습니다. 확인 되었습니다 고객님. 성함이나 휴대폰 번호 알려주세요. 공공공이고 공공공 공공공 공공공입니다. 대출 한도가 다 대출건과 합산이 되어 산정됩니다. 필요하신 자금은 얼마신가요? 3,000만원 정도 필요합니다. 대출 한도는 스마트폰 비대면 대출 메뉴를 이용하시거나 지점을 방문해 주시면 진행 가능합니다. 문자로 비대면 대출 방법을 보내주실 수 있나요? 네, 이 번호로 보내주세요.
✅ 실제 라벨: 정상 통화
🔍 예측 라벨: 보이스 피싱 (98.99% 확신도)

📞 샘플 2
▶ 원본 텍스트: 신용대출은 인터넷으로 신청할 수 있어요? 아니요 방문하셔야 합니다 아 그렇군요 아무 노겹이다 가면 되나요? 네 맞습니다 서류는 뭘 가져가면 되죠? 신분증과 소득 확인 서류 가져가시면 됩니다 소득 확인 서류는 뭐가 있어요? 혹시 기존의 대출 내역이 있으면 불리한가요? 네 기존 대출 때문에 불가능할 수도 있습니다. 여기서 미리 말해 주실 수는 없어요? 네 영업점 방문 후 상담 하시면 됩니다. 서류는 말씀해 주신 것만 가져가면 되는 거죠? 네 추후 대출 과정에서 필요 서류가 더 많아지게 되면 있을 수 있습니다. 그건 나중에 안내해주시는 거예요? 네, 맞습니다. 오늘 영업시가 몇 시까지예요? 6시입니다. 감사합니다.
✅ 실제 라벨: 정상 통화
🔍 예측 라벨: 보이스 피싱 (99.03% 확신도)
